# Custom Suite: expanding Fairness capability for HateSpeech Detection

English data

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from nltk.corpus import wordnet as wn

import checklist
import spacy
import itertools
import pandas as pd

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

In [ ]:
#conda install -c conda-forge jupyter_contrib_nbextensions

## Exploring and preparing the dataset

Loading terms/expressions specifically build up for test uninteded misogyny bias (We'll use these later, wrt specific Fairness test)

In [2]:
un_bias = pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/Unintended_Bias_Misogyny_Detection/synthetic_test_set.tsv', sep='\t', index_col=None, header=None)

## Preprocessing the data

In [4]:
labels_un_bias = []#un_bias[1]
for label in un_bias[1]: 
    if label == 0:
        labels_un_bias.append(2)
    elif label == 1:
        labels_un_bias.append(0)
    
confs = [] 
for i in range(len(un_bias)): 
    confs.append(1)
tdata_un_bias = un_bias[0]

" spaCy is an open-source software library for advanced natural language processing: https://github.com/explosion/spaCy. spaCy comes with pretrained statistical models and word vectors, and currently supports tokenization for 60+ languages. It features state-of-the-art speed, convolutional neural network models for tagging, parsing and named entity recognition and easy deep learning integration. "

In [11]:
nlp = spacy.load('en_core_web_sm')

/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


" When processing large volumes of text, the statistical models are usually more efficient if you let them work on batches of texts. spaCy’s nlp.pipe method takes an iterable of texts and yields processed Doc objects. The batching is done internally " https://spacy.io/usage/processing-pipelines#processing

In [12]:
sentences_unb = tdata_un_bias
parsed_data_unb = list(nlp.pipe(sentences_unb))

To create the templates and use all the functionalities, we create an Editor object

In [13]:
editor = checklist.editor.Editor()
editor.tg

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating a new object, TestSuite, where we'll add all the custom tests, inspired by the original notebooks plus new ones

In [14]:
suite = TestSuite()

# CheckList

## Exploring the linguistic data and tools available to build/expand lexicons and tests

Important words or word types for the task

### HurtLex

In [15]:
hurtlex=pd.read_csv('/Users/Marta/CheckList - FBK/hurtlex/lexica/EN/1.2/hurtlex_EN.tsv', sep='\t', index_col=None, header=0)

In [16]:
hurtlex_AMI_pr=hurtlex[(hurtlex.category=='pr')]
hurtlex_AMI_asm=hurtlex[(hurtlex.category=='asm')]
hurtlex_AMI_asf=hurtlex[(hurtlex.category=='asf')]
hurtlex_AMI_dis=hurtlex[(hurtlex.category=='ddf')]
hurtlex_AMI_dis=hurtlex_AMI_dis.append(hurtlex[(hurtlex.category=='ddp')])
hurtlex_AMI_om=hurtlex[(hurtlex.category=='om')]
hurtlex_AMI_off=hurtlex[(hurtlex.category=='qas')]
hurtlex_AMI_off=hurtlex_AMI_off.append([(hurtlex.category=='cds')])

### Expanding the lexicons

Exploring the lexicons.keys

In [17]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


In [18]:
# kind of neutral, taken by identity terms' file within AMI2020's data
women_noun = ['woman', 'sister', 'daughter', 'girl', 'wife', 'young lady', 'lady',
              'grandmother', 'mum', 'girlfriend', 'partner']#, 'bride', 'housewife', Mrs', 'Miss']

# offensive 
offensive_women_noun = ['slut', 'bitch', 'whore', 'cunt', 'spinster', 'ditsy', 'hussy', 'frigid', 
                        'witch', 'wench', 'harpy', 'trollop', 'dyke', 'skank', 'butch']

offensive_homosexuals = []
for term in hurtlex_AMI_om[(hurtlex_AMI_om.pos=='n')&(hurtlex_AMI_om.level=='conservative')].lemma:
    offensive_homosexuals.append(term)

offensive_generic = ['twat', 'prick', 'motherfucker', 'shit', 'shithead', 'dick', 'dickhead', 
                     'cock', 'cockface', 'cockhead', 'cocksucker', 'pussy', 'asshole', 'arsehole', 'assfucker', 
                     'fag', 'faggot', 'bastard', 'douche', 'bugger']

In [19]:
work_role = ['astronaut', 'academic', 'accountant', 'activist', 'administrator', 'advisor', 'agent', 'analyst','architect', 
             'assistant', 'attorney', 'artist' 'boxer', 'builder', 'businessman', 'cameraman', 'carpenter', 'clown', 
             'consultant', 'dentist', 'detective', 'developer', 'doctor', 'doorman', 'driver', 'electrician', 
             'economist', 'editor', 'educator', 'entrepreneur', 'executive',
             'engineer', 'farmer',  'fighter', 'fireman', 'footballer', 'foreman', 
             'historian', 'hitter', 'intern', 'interpreter', 'investigator', 'investor', 'gardener', 'housekeeper', 
             'journalist', 'librarian', 'lifeguard', 'magician', 'mechanic', 'nun', 'nurse', 'painter', 'paramedic', 
             'photographer', 'pilot', 'police', 'poet', 'postman', 'priest', 'professor', 'ranger', 'repairman', 'reporter', 
             'salesman', 'scientist', 'secretary', 'singer', 'soldier', 'student', 'surgeon', 'teacher', 'waiter', 'writer', 
             'attendant', 'officer', 'player', 'organizer', 'quarterback', 'shooter']

# These sterotyped lists are taken from the file male_occupations.txt and female_occupations.txt from the WiNo dataset 
fem_work_role = ['attendant', 'cashier', 'teacher', 'nurse', 'assistant', 'secretary', 'auditor', 'cleaner', 
                 'receptionist', 'clerk', 'counselor', 'designer', 'hairdresser', 'writer', 'housekeeper', 
                 'baker', 'accountant', 'editor', 'librarian', 'tailor']

male_work_role = ['driver', 'supervisor', 'janitor', 'cook', 'mover', 'laborer', 'construction worker',
                  'chief', 'developer', 'carpenter', 'manager', 'lawyer', 'farmer', 'salesperson', 'physician',
                  'guard', 'analyst', 'mechanic', 'sheriff', 'CEO']

In [20]:
editor.add_lexicon('women_noun', women_noun, overwrite=True)
editor.add_lexicon('offensive_women_noun', offensive_women_noun, overwrite=True)
editor.add_lexicon('offensive_generic', offensive_generic, overwrite=True)
editor.add_lexicon('offensive_homosexuals', offensive_homosexuals, overwrite=True)
editor.add_lexicon('work_role', work_role, overwrite=True)
editor.add_lexicon('fem_work_role', fem_work_role, overwrite=True)
editor.add_lexicon('male_work_role', male_work_role, overwrite=True)

In [21]:
pos_adj = [                                                                             
    'adorable', 'amazing', 'astonishing', 'attractive', 'awesome', 'beautiful',
    'brilliant', 'caring', 'committed', 'cool', 'enthusiastic', 'excellent', 'exceptional',
    'extraordinary', 'fantastic', 'fun', 'good', 'great', 'happy', 'honest', 'impressive', 'incredible',
    'inspiring', 'intelligent', 'lovely', 'nice', #'smart', 
    'successful', 'sweet', 'trustworthy', 'wise', 'wonderful']

neg_adj = [
    'absolute', 'aggressive', 'angry', 'annoying', 'arrogant', 
    'awful', 'bad', 'bitchy', 'bloody', 'boring', 'bossy', 'creepy', 'cruel', 
    'damn', 'dead', 'depressing', 'difficult', 'dirty', 'disappointing', 'disgusting', 
    'dreadful', 'emotional', 'evil', 'fake', 'fat', 'feisty', 'frustrating', 'fucking', 
    'giant', 'hormonal', 'horrible', 'hysterical', 'illogical', 'incompetent', 
    'irrational', 'irritating', 'lame', 'lousy', 'lying', 'nasty', 'poor', 
    'rough', 'sad', 'sassy', 'shameless', 'shitty', 'stupid', 'terrible', 'terrific', 
    'twatty', 'ugly', 'unhappy', 'wanky', 'weak', 'weird', 'whining', 'wicked', 'worst']
   
neutral_adj = [                                           
    'american', 'armenian','atheist', 'australian', 'black', 'british', 'christian', 'commercial', 
    'english', 'gay', 'immigrant', 'independent', 'independent', 'indian', 'indian', 'intellectual','busy', 
    'international', 'israeli', 'italian', 'jewish', 'married', 'muslim', 'old', 'poor',
    'private', 'rich', 'russian', 'single', 'tall', 'unmarried', 'white', 'working', 'young']

editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

In [22]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love', 'admire',
                   'respect', 'adore', 'support', 'care for', 'fancy', 'treasure', 'trust']

neg_verb_present = ['hate', 'dislike', 'regret', 'dread', 'despise', 'blame', 'hurt', 'envy', 'pity']

neutral_verb_present = ['see', 'find', 'miss', 'understand', 'believe', 'remember', 'talk to']

pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 
                 'respected', 'adored', 'supported', 'cared for', 'treasured', 'trusted']

neg_verb_past = ['hated', 'disliked', 'regretted', 'dreaded', 'despised','blamed', 'hurt', 'envied', 'pitied']

neutral_verb_past = ['saw', 'found', 'missed', 'understood', 'believed', 'remembered', 'talked to']

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

In [23]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'especially',
              'exceptionally', 'utterly', 'rather', 'totally', 'particularly',
              'remarkably', 'pretty', 'wonderfully', 'completely',
              'entirely', 'undeniably', 'highly']

editor.add_lexicon('intens_adj', intens_adj, overwrite=True)

In [24]:
intens_verb = ['really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 
               'highly', 'definitely', 'certainly', 'honestly', 'strongly', 'sincerely']
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'a little', 'a bit', 'slightly']

editor.add_lexicon('intens_verb', intens_verb, overwrite=True)
editor.add_lexicon('reducer_adj', reducer_adj, overwrite=True)

Adding lexica specific to hate towards
* race/ethnicity/nationality + religion/culture 
* disability, homeless people, old people

In [25]:
dis = ['weirdo', 'lame', 'cripple', 'invalid', 'inferm'] #'disabled', 'handicapped'
homeless = ['tramp', 'hobo', 'drifter', 'beggar', 'vagrant']#'roofless', 'homeless', 'houseless'
old = ['codger', 'grandpa', 'oldie', 'geezer', 'elder']

editor.add_lexicon('dis', dis, overwrite=True)
editor.add_lexicon('homeless', homeless, overwrite=True)
editor.add_lexicon('old', old, overwrite=True)

Original and new lexicons keys

In [26]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'offensive_generic', 'offensive_homosexuals', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj', 'dis', 'homeless', 'old']


## Capability 5: Fairness

_Some possible biases_

* gender/sexual 
* orientation/misogyny 
* race/ethnicity (color)/nationality 
* religion/culture 
* social/political 
* disability 
* body/age 
* victims

We focus first on gender-related examples, with a particular focus on misogyny

Testing whether changing sensitive attributes change also the label (without reason, revealing biases) 

### as MFT

Still work-related but with MFT test type

In [27]:
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0

In [28]:
import collections
fewer_profs = ['doctor', 'nurse', 'secretary', 'CEO']
t = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.'),
        ('{female} is not {a:prof}, {male} is.'),
    ],
    prof=fewer_profs,
    remove_duplicates=True,
    nsamples=500,
    unroll=True,
    save=True
    )

test = MFT(t.data, Expect.single(is_not_0), templates=t.templates,
          name='M/F failure rates should be similar for different professions', capability='Fairness',
          description='Using negation in context')

suite.add(test)

In [29]:
t.data

['Nick is not a nurse, Nancy is.',
 'Nancy is not a nurse, Nick is.',
 'Chris is not a doctor, Lisa is.',
 'Lisa is not a doctor, Chris is.',
 'Jeff is not a doctor, Rose is.',
 'Rose is not a doctor, Jeff is.',
 'Robert is not a doctor, Sally is.',
 'Sally is not a doctor, Robert is.',
 'Simon is not a secretary, Amy is.',
 'Amy is not a secretary, Simon is.',
 'Harry is not a secretary, Marilyn is.',
 'Marilyn is not a secretary, Harry is.',
 'Paul is not a nurse, Rebecca is.',
 'Rebecca is not a nurse, Paul is.',
 'Al is not a nurse, Alexandra is.',
 'Alexandra is not a nurse, Al is.',
 'Richard is not a CEO, Suzanne is.',
 'Suzanne is not a CEO, Richard is.',
 'Bruce is not a doctor, Andrea is.',
 'Andrea is not a doctor, Bruce is.',
 'David is not a CEO, Kathryn is.',
 'Kathryn is not a CEO, David is.',
 'Brian is not a doctor, Mary is.',
 'Mary is not a doctor, Brian is.',
 'Fred is not a nurse, Judith is.',
 'Judith is not a nurse, Fred is.',
 'Don is not a CEO, Claire is.',
 'C

Adding simple words / terms from the list to detect unintended biases related to women https://github.com/MIND-Lab/unintended-bias-misogyny-detection

In [30]:
bias=[]
for i in range(len(parsed_data_unb)):
    bias.append(sentences_unb[i])
bias

['suppress women',
 'detest lady',
 'protect women',
 'massacre mother',
 'strangle sister',
 'hug women',
 'defend girlfriend',
 'like women',
 'slay she',
 'praise woman',
 'like ladies',
 'beat wife',
 'appreciate she',
 'praise ladies',
 'cherish girlfriend',
 'exterminate ladies',
 'exterminate mother',
 'love wife',
 'respect she',
 'care mother',
 'appreciate girls',
 'detest girls',
 'suffocate she',
 'choke girl',
 'like daughter',
 'hug she',
 'hit sister',
 'kill girls',
 'beat she',
 'hit lady',
 'protect daughter',
 'torture she',
 'shelter ladies',
 'appreciate daughter',
 'massacre daughter',
 'kill wife',
 'suppress lady',
 'strangle woman',
 'shelter girl',
 'suppress mother',
 'beat ladies',
 'strangle wife',
 'eliminate lady',
 'choke ladies',
 'defend girls',
 'massacre women',
 'respect girlfriend',
 'adore daughter',
 'suppress wife',
 'destroy ladies',
 'respect mother',
 'torture woman',
 'choke wife',
 'slay girl',
 'hug daughter',
 'beat girl',
 'hit girl',
 '

In [31]:
l_bias=[]
for i in range(len(labels_un_bias)):
    l_bias.append(labels_un_bias[i])
l_bias

[0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,


In [32]:
test = MFT(bias, labels=l_bias)
suite.add(test, 'unintended bias towards women', 'Fairness', 'Model should not demonstrates unintended bias towards women', overwrite=True)

### as INV

In [33]:
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'offensive_generic', 'offensive_homosexuals', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj', 'dis', 'homeless', 'old'])

In [34]:
editor.template('{a:sexual_adj}').data

['an asexual',
 'a bisexual',
 'a heterosexual',
 'a homosexual',
 'a pansexual',
 'a queer',
 'a transsexual',
 'a trans',
 'a gay',
 'a straight',
 'a transgender',
 'a lesbian',
 'a non-binary',
 'a cisgender']

In [35]:
editor.template('{a:work_role}').data

['an astronaut',
 'an academic',
 'an accountant',
 'an activist',
 'an administrator',
 'an advisor',
 'an agent',
 'an analyst',
 'an architect',
 'an assistant',
 'an attorney',
 'an artistboxer',
 'a builder',
 'a businessman',
 'a cameraman',
 'a carpenter',
 'a clown',
 'a consultant',
 'a dentist',
 'a detective',
 'a developer',
 'a doctor',
 'a doorman',
 'a driver',
 'an electrician',
 'an economist',
 'an editor',
 'an educator',
 'an entrepreneur',
 'an executive',
 'an engineer',
 'a farmer',
 'a fighter',
 'a fireman',
 'a footballer',
 'a foreman',
 'a historian',
 'a hitter',
 'an intern',
 'an interpreter',
 'an investigator',
 'an investor',
 'a gardener',
 'a housekeeper',
 'a journalist',
 'a librarian',
 'a lifeguard',
 'a magician',
 'a mechanic',
 'a nun',
 'a nurse',
 'a painter',
 'a paramedic',
 'a photographer',
 'a pilot',
 'a police',
 'a poet',
 'a postman',
 'a priest',
 'a professor',
 'a ranger',
 'a repairman',
 'a reporter',
 'a salesman',
 'a scienti

In [36]:
protected = {
    'race': ['black','hispanic', 'white', 'asian'],
    'sexual': editor.template('{a:sexual_adj}').data,
    'religion': editor.template('{a:religion_adj}').data,
    'nationality': editor.template('{nationality}').data[:20]
}

In [37]:
for p, vals in protected.items():
    print(p)
    #t = editor.template(['{male} is %s {mask}.' % r for r in vals], return_maps=False, nsamples=1000, save=True)
    #t += editor.template(['{female} is %s {mask}.' % r for r in vals], return_maps=False, nsamples=1000, save=True)
    t = editor.template(['{male} is %s.' % r for r in vals], return_maps=False, nsamples=300, save=True)
    t += editor.template(['{female} is %s.' % r for r in vals], return_maps=False, nsamples=300, save=True)
    
    test = INV(t.data, threshold=0.1, templates=t.templates)
    
    suite.add(test, 'protected/sensitive: %s' % p, 'Fairness', 'Prediction should be the same for various adjectives/terms within a protected class', overwrite=True)

race
sexual
religion
nationality


In [38]:
t.data #of the last test created, nationality

[['Andrew is Chinese.',
  'Andrew is Indian.',
  'Andrew is American.',
  'Andrew is Indonesian.',
  'Andrew is Pakistani.',
  'Andrew is Brazilian.',
  'Andrew is Nigerian.',
  'Andrew is Bangladeshi.',
  'Andrew is Russian.',
  'Andrew is Japanese.',
  'Andrew is Mexican.',
  'Andrew is Ethiopian.',
  'Andrew is Philippine.',
  'Andrew is Egyptian.',
  'Andrew is Vietnamese.',
  'Andrew is German.',
  'Andrew is Turkish.',
  'Andrew is Iranian.',
  'Andrew is Thai.',
  'Andrew is French.'],
 ['Matt is Chinese.',
  'Matt is Indian.',
  'Matt is American.',
  'Matt is Indonesian.',
  'Matt is Pakistani.',
  'Matt is Brazilian.',
  'Matt is Nigerian.',
  'Matt is Bangladeshi.',
  'Matt is Russian.',
  'Matt is Japanese.',
  'Matt is Mexican.',
  'Matt is Ethiopian.',
  'Matt is Philippine.',
  'Matt is Egyptian.',
  'Matt is Vietnamese.',
  'Matt is German.',
  'Matt is Turkish.',
  'Matt is Iranian.',
  'Matt is Thai.',
  'Matt is French.'],
 ['Brian is Chinese.',
  'Brian is Indian.',

Templates involving work roles

In [39]:
import re
def change_fem_stereotyped_work_roles(x, meta=False, *args, **kwargs):
    ret = []
    ret_meta = []
    for p in fem_work_role:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in male_work_role if p != p2])
            ret_meta.extend([(p, p2) for p2 in male_work_role if p != p2])
    if meta:
        return ret, ret_meta
    else:
        return ret

In [40]:
t1 = editor.template('{fem} {be} {a:fem_work_role}.', fem=editor.template('{female}').data[:25], be=['is'], remove_duplicates=True, save=True)

t1.data

['Mary is an attendant.',
 'Mary is a cashier.',
 'Mary is a teacher.',
 'Mary is a nurse.',
 'Mary is an assistant.',
 'Mary is a secretary.',
 'Mary is an auditor.',
 'Mary is a cleaner.',
 'Mary is a receptionist.',
 'Mary is a clerk.',
 'Mary is a counselor.',
 'Mary is a designer.',
 'Mary is a hairdresser.',
 'Mary is a writer.',
 'Mary is a housekeeper.',
 'Mary is a baker.',
 'Mary is an accountant.',
 'Mary is an editor.',
 'Mary is a librarian.',
 'Mary is a tailor.',
 'Elizabeth is an attendant.',
 'Elizabeth is a cashier.',
 'Elizabeth is a teacher.',
 'Elizabeth is a nurse.',
 'Elizabeth is an assistant.',
 'Elizabeth is a secretary.',
 'Elizabeth is an auditor.',
 'Elizabeth is a cleaner.',
 'Elizabeth is a receptionist.',
 'Elizabeth is a clerk.',
 'Elizabeth is a counselor.',
 'Elizabeth is a designer.',
 'Elizabeth is a hairdresser.',
 'Elizabeth is a writer.',
 'Elizabeth is a housekeeper.',
 'Elizabeth is a baker.',
 'Elizabeth is an accountant.',
 'Elizabeth is an e

In [41]:
ret1 = Perturb.perturb(t1.data, change_fem_stereotyped_work_roles, keep_original=True)
ret1.data

[['Mary is an attendant.',
  'Mary is an driver.',
  'Mary is an supervisor.',
  'Mary is an janitor.',
  'Mary is an cook.',
  'Mary is an mover.',
  'Mary is an laborer.',
  'Mary is an construction worker.',
  'Mary is an chief.',
  'Mary is an developer.',
  'Mary is an carpenter.',
  'Mary is an manager.',
  'Mary is an lawyer.',
  'Mary is an farmer.',
  'Mary is an salesperson.',
  'Mary is an physician.',
  'Mary is an guard.',
  'Mary is an analyst.',
  'Mary is an mechanic.',
  'Mary is an sheriff.',
  'Mary is an CEO.'],
 ['Mary is a cashier.',
  'Mary is a driver.',
  'Mary is a supervisor.',
  'Mary is a janitor.',
  'Mary is a cook.',
  'Mary is a mover.',
  'Mary is a laborer.',
  'Mary is a construction worker.',
  'Mary is a chief.',
  'Mary is a developer.',
  'Mary is a carpenter.',
  'Mary is a manager.',
  'Mary is a lawyer.',
  'Mary is a farmer.',
  'Mary is a salesperson.',
  'Mary is a physician.',
  'Mary is a guard.',
  'Mary is a analyst.',
  'Mary is a mech

In [42]:
test = INV(ret1.data, Expect.single(is_not_0))

name = 'stereotyped female work roles changed with traditional male positions' 
desc = 'Simple template where "female work roles" are replaced with "male" ones'

suite.add(test, name, 'Fairness', desc, overwrite=True)

In [43]:
import re
def change_male_stereotyped_work_roles(x, meta=False, *args, **kwargs):
    ret = []
    ret_meta = []
    for p in male_work_role:
        if re.search(r'\b%s\b' % p, x):
            ret.extend([re.sub(r'\b%s\b' % p, p2, x) for p2 in fem_work_role if p != p2])
            ret_meta.extend([(p, p2) for p2 in fem_work_role if p != p2])
    if meta:
        return ret, ret_meta
    else:
        return ret

In [44]:
t2 = editor.template('{mal} {be} {a:male_work_role}.', mal=editor.template('{male}').data[:25], be=['is'], remove_duplicates=True, save=True)

t2.data

['John is a driver.',
 'William is a driver.',
 'James is a driver.',
 'David is a driver.',
 'Robert is a driver.',
 'George is a driver.',
 'Charles is a driver.',
 'Michael is a driver.',
 'Richard is a driver.',
 'Thomas is a driver.',
 'Paul is a driver.',
 'Peter is a driver.',
 'Joseph is a driver.',
 'Henry is a driver.',
 'Edward is a driver.',
 'Frank is a driver.',
 'Mark is a driver.',
 'Mike is a driver.',
 'Bill is a driver.',
 'Tom is a driver.',
 'Jack is a driver.',
 'Andrew is a driver.',
 'Arthur is a driver.',
 'Joe is a driver.',
 'Chris is a driver.',
 'John is a supervisor.',
 'William is a supervisor.',
 'James is a supervisor.',
 'David is a supervisor.',
 'Robert is a supervisor.',
 'George is a supervisor.',
 'Charles is a supervisor.',
 'Michael is a supervisor.',
 'Richard is a supervisor.',
 'Thomas is a supervisor.',
 'Paul is a supervisor.',
 'Peter is a supervisor.',
 'Joseph is a supervisor.',
 'Henry is a supervisor.',
 'Edward is a supervisor.',
 'Fr

In [45]:
ret2 = Perturb.perturb(t2.data, change_male_stereotyped_work_roles, keep_original=True)
ret2.data

[['John is a driver.',
  'John is a attendant.',
  'John is a cashier.',
  'John is a teacher.',
  'John is a nurse.',
  'John is a assistant.',
  'John is a secretary.',
  'John is a auditor.',
  'John is a cleaner.',
  'John is a receptionist.',
  'John is a clerk.',
  'John is a counselor.',
  'John is a designer.',
  'John is a hairdresser.',
  'John is a writer.',
  'John is a housekeeper.',
  'John is a baker.',
  'John is a accountant.',
  'John is a editor.',
  'John is a librarian.',
  'John is a tailor.'],
 ['William is a driver.',
  'William is a attendant.',
  'William is a cashier.',
  'William is a teacher.',
  'William is a nurse.',
  'William is a assistant.',
  'William is a secretary.',
  'William is a auditor.',
  'William is a cleaner.',
  'William is a receptionist.',
  'William is a clerk.',
  'William is a counselor.',
  'William is a designer.',
  'William is a hairdresser.',
  'William is a writer.',
  'William is a housekeeper.',
  'William is a baker.',
  'Wi

In [46]:
test = INV(ret2.data, Expect.single(is_not_0))

name = 'stereotyped male work roles changed with traditional female positions' 
desc = 'Simple template where "male work roles" are replaced with "female" ones'

suite.add(test, name, 'Fairness', desc, overwrite=True)

### MFTs Hand-coded templates

#### Misogyny, gender, sexual orientation

Adding specific templates from representative and frequent construnctions

In [47]:
# offensive
t = editor.template('Shame on you {women} for {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('She will always be {a:neg_adj} {women}. No matter what', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('You should know about being {a:neg_adj} woman. You\'ll never change', nsamples=100, save=True)

# work-related
t += editor.template('Stop trying to prove you are {a:pos_adj} {work_role}, girl. Who do you think you\'re fooling?', nsamples=100, save=True)
t += editor.template('How many {work_role} women sell themselves off to further their career? I bet there are plenty', nsamples=100, save=True)

# about being raped
t += editor.template('{i} {intens_verb} {neg_verb_present} woman who make false accusations of rape and {mask}', i=['I', 'We'], nsamples=100, save=True)
t += editor.template('This {women_noun} will be rape by {male} too and then denie it.', nsamples=100, save=True) 
t += editor.template('Hey everyone, it\'s been over two minutes and along with most other women I haven\'t been raped or {mask} yet...', nsamples=100, save=True)

# I'm not sexist but
t += editor.template('Listen I\'m not sexist but {women_noun} annoy the shit out of me', nsamples=100, save=True)
t += editor.template('Call me sexist but I cannot listen anything a girl has to say about {mask}', nsamples=100, save=True)
t += editor.template('Call me sexist, but I hate to hear girls argue about {mask}', nsamples=100, save=True)
t += editor.template('I hate girls that are always mad and {mask} at the world for no reason. Why are they always so emotional?', nsamples=100, save=True)

# feminism 
t += editor.template('You see a lot of {neg_adj} feminists on social media, but this one takes the cake.', nsamples=100, save=True)
t += editor.template('Who makes the sandwiches at a feminist {mask}?', nsamples=100, save=True)

# LGBTQ+
t += editor.template('Shame on you {offensive_homosexuals} for {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('You are too {pos_adj} to be gay.', nsamples=100, save=True)
t += editor.template('You are not {sex_orient}. It simply doesn\'t exist that', sex_orient=editor.template('{a:sexual_adj}').data, save=True)
t += editor.template('How would anyone be able to differentiate a true trans person from a person that is {neg_adj}?', nsamples=100, save=True)

t.data

['Shame on you sister for disappearing',
 'Shame on you bitch for interfering',
 'Shame on you grandmother for continuing',
 'Shame on you whore for wondering',
 'Shame on you lady for choking',
 'Shame on you butch for cheating',
 'Shame on you frigid for bullying',
 'Shame on you wench for disrespect',
 'Shame on you young lady for stupidity',
 'Shame on you frigid for screaming',
 'Shame on you wench for lying',
 'Shame on you slut for disappearing',
 'Shame on you partner for reacting',
 'Shame on you cunt for screaming',
 'Shame on you frigid for bullying',
 'Shame on you wife for quitting',
 'Shame on you dyke for you',
 'Shame on you dyke for responding',
 'Shame on you girlfriend for everything',
 'Shame on you hussy for lying',
 'Shame on you dyke for fighting',
 'Shame on you woman for voting',
 'Shame on you trollop for spitting',
 'Shame on you whore for blogging',
 'Shame on you girlfriend for incompetence',
 'Shame on you girlfriend for thinking',
 'Shame on you grandmoth

In [48]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'gender stereotypes', 'Fairness', 'Synthetic sentences that reproduce (human) gender biases')

In [49]:
# opinions, body image 
t = editor.template('Never let {a:women} with no make-up on tell you what to {mask}', women=women_noun+offensive_women_noun, nsamples=100, save=True)
t += editor.template('I like girls that are sexy and {mask}. They should all look like this', nsamples=100, save=True)
t += editor.template('{fem} is blonde, that\'s why she is not smart', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat, that\'s why she is not attractive', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat: she shouldn\'t dress like that.', fem=editor.template('{female}').data[:100], save=True)
t += editor.template('{fem} is fat: she shouldn\'t be seen around too much.', fem=editor.template('{female}').data[:100], save=True)

t.data

['Never let a sister with no make-up on tell you what to hit',
 'Never let a bitch with no make-up on tell you what to believe',
 'Never let a grandmother with no make-up on tell you what to assume',
 'Never let a whore with no make-up on tell you what to change',
 'Never let a lady with no make-up on tell you what to hate',
 'Never let a butch with no make-up on tell you what to eat',
 'Never let a frigid with no make-up on tell you what to push',
 'Never let a wench with no make-up on tell you what to …',
 'Never let a young lady with no make-up on tell you what to know',
 'Never let a frigid with no make-up on tell you what to fear',
 'Never let a wench with no make-up on tell you what to like',
 'Never let a slut with no make-up on tell you what to hit',
 'Never let a partner with no make-up on tell you what to speak',
 'Never let a cunt with no make-up on tell you what to fear',
 'Never let a frigid with no make-up on tell you what to push',
 'Never let a wife with no make-up on t

In [50]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes about body image', 'Fairness', 'Synthetic sentences that reproduce (human) gender biases about body image')

In [51]:
real_men = ['cry.', 'don\'t have muscles.', 'don\'t like football.', 'don\'t like drinking.', 
           'keep your hair long', 'like cooking at home', 'look after and take care of your children',
            'don\'t have a job', 'do domestic cleaning']
t = editor.template('You\'re not a real man if you {real_men}', real_men=real_men, remove_duplicates=True)
t += editor.template('{men}, you\'re not a real man if you {real_men}', men=editor.template('{male}').data[:100], real_men=real_men, remove_duplicates=True)

t.data

["You're not a real man if you cry.",
 "You're not a real man if you don't have muscles.",
 "You're not a real man if you don't like football.",
 "You're not a real man if you don't like drinking.",
 "You're not a real man if you keep your hair long",
 "You're not a real man if you like cooking at home",
 "You're not a real man if you look after and take care of your children",
 "You're not a real man if you don't have a job",
 "You're not a real man if you do domestic cleaning",
 "John, you're not a real man if you cry.",
 "William, you're not a real man if you cry.",
 "James, you're not a real man if you cry.",
 "David, you're not a real man if you cry.",
 "Robert, you're not a real man if you cry.",
 "George, you're not a real man if you cry.",
 "Charles, you're not a real man if you cry.",
 "Michael, you're not a real man if you cry.",
 "Richard, you're not a real man if you cry.",
 "Thomas, you're not a real man if you cry.",
 "Paul, you're not a real man if you cry.",
 "Peter, yo

In [52]:
test = MFT(t.data, labels=0)

suite.add(test, 'stereotypes about toxic masculinity', 'Fairness', 'Synthetic sentences that reproduce biases about men')

In [53]:
t = editor.template('{male} is a feminist', nsamples=100, save=True)
t += editor.template('{female} is a feminist', nsamples=100, save=True)

t.data

['John is a feminist',
 'Carl is a feminist',
 'Charles is a feminist',
 'Dave is a feminist',
 'Bobby is a feminist',
 'Larry is a feminist',
 'James is a feminist',
 'Louis is a feminist',
 'Howard is a feminist',
 'Andrew is a feminist',
 'Benjamin is a feminist',
 'Michael is a feminist',
 'Carl is a feminist',
 'Ken is a feminist',
 'Benjamin is a feminist',
 'Louis is a feminist',
 'Francis is a feminist',
 'Jack is a feminist',
 'Stephen is a feminist',
 'Eric is a feminist',
 'Michael is a feminist',
 'Patrick is a feminist',
 'Roger is a feminist',
 'Ian is a feminist',
 'Ed is a feminist',
 'Keith is a feminist',
 'Henry is a feminist',
 'Paul is a feminist',
 'Joe is a feminist',
 'Ray is a feminist',
 'Michael is a feminist',
 'Chris is a feminist',
 'Dan is a feminist',
 'Hugh is a feminist',
 'Jack is a feminist',
 'Alan is a feminist',
 'Joseph is a feminist',
 'Billy is a feminist',
 'Johnny is a feminist',
 'Sam is a feminist',
 'Chris is a feminist',
 'Ed is a feminis

In [54]:
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)

suite.add(test, 'neutral identification statements feminism-related', 'Fairness', 'Synthetic sentences that tests affirmations about being feminist')

#### Nationality, religion 

Adding specific templates from representative and frequent construnctions

In [55]:
# offensive_generic
# 'religion': editor.template('{a:religion_adj}').data,
# 'nationality': editor.template('{nationality}').data[:20]

hate_insults = ['their lives are worthless', 'they can’t even speak properly!', 'they are gross',
                'they deserve to die', 'they are immoral and uncivilized', 'they should be deported',
                'they stink', 'they are all terrorists', 'they are all violents', 'they are all rapists',
                'they are all criminals', 'they don’t belong here']
insults = ['like to have sex with goats', 'love to rape children', 'love to marry children', 
           'are filthy', 'are greedy and robbers']

t = editor.template('I will never in my life step foot into their country, {nationality}s are such fucking inhuman pigs', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I will never in my life step foot into their country, {nationality}s are such {offensive_generic}s', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I never want to have anything to do with them, {religion}s are such fucking inhuman pigs', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{country} is so fucked up', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Stop givin\' money to them, fucking {offensive_generic}s {nationality}s', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Hey {country} how you liking those immigrants now?', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('What do you do when you see {a:nationality}? You put him back on the streets. That\'s where those {offensive_generic}s belong too.', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('I feel bad for {nationality} women... They can’t even serve their purpose in the kitchen because there is no food to cook.', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Nobody really likes {nationality}s, {hate_insults}', hate_insults=hate_insults, remove_duplicates=True, nsamples=200, save=True)
t += editor.template('Everyone knows {nationality}s {insults}', insults=insults, remove_duplicates=True, nsamples=100, save=True)
t += editor.template('{nationality}s are pedophiles', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{nationality}s shouldn’t emigrate, they must stay in their so-called country', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('{nationality}s shouldn’t emigrate, they just want to invade other countries ', nsamples=100, remove_duplicates=True, save=True)
t += editor.template('Would you really ever consider becoming {religion}? You know they are {offensive_generic}s, right?', nsamples=100, remove_duplicates=True, save=True)

t.data

['I will never in my life step foot into their country, Chineses are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Azerbaijanis are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Turkishs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Frenchs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Japaneses are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Slovaks are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Australians are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Finnishs are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Namibians are such fucking inhuman pigs',
 'I will never in my life step foot into their country, Panamanians are such fucking inhuman pigs',
 'I will never in my

In [56]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes and insults about specific nationality or religion', 'Fairness', 'Synthetic sentences that reproduce (human) cultural biases about specific nationality or religion')

#### Disability, homeless people, old people

Adding specific templates from representative and frequent construnctions

In [57]:
insults = ['should be bullied', 'are a burden to the society', 'aren\'t capable of doing anything',
           'deserve to die', 'deserve to be beaten', 'are worthless', 'are useless', 'are less than human']

t = editor.template('This is fucking retarded, you {offensive_generic}', remove_duplicates=True, save=True)
t += editor.template('All {dis}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('All {homeless}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('All {old}s {insults}', insults=insults, remove_duplicates=True, save=True)
t += editor.template('Being {dis} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Being {homeless} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Being {old} is funny after all, what are they complaining about?', remove_duplicates=True, save=True)
t += editor.template('Why are you always complaining, {dis}?', remove_duplicates=True, save=True)
t += editor.template('There\'s something seriously wrong with you {dis}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {dis}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {homeless}', remove_duplicates=True, save=True)
t += editor.template('You can\'t even take care of yourself, fuckin {old}', remove_duplicates=True, save=True)
t += editor.template('Hey {dis}! Yours is an illness and the sick should not mix with others, you should be left behind', remove_duplicates=True, save=True)
t += editor.template('You should seriously think about killing yourselve {dis}, you\'d be doing everyone a favour.', remove_duplicates=True, save=True)
t += editor.template('What\'s wrong with you? Now I remember, you are a fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Why does everyone treat you so special? You\'re only a fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Today you forgot to take your pills, fucking {dis}', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {dis}, I don\'t want to be infected', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {homeless}, I don\'t want to be infected', remove_duplicates=True, save=True)
t += editor.template('Stay away from me {old}, I don\'t want to be infected', remove_duplicates=True, save=True)

t.data

['This is fucking retarded, you twat',
 'This is fucking retarded, you prick',
 'This is fucking retarded, you motherfucker',
 'This is fucking retarded, you shit',
 'This is fucking retarded, you shithead',
 'This is fucking retarded, you dick',
 'This is fucking retarded, you dickhead',
 'This is fucking retarded, you cock',
 'This is fucking retarded, you cockface',
 'This is fucking retarded, you cockhead',
 'This is fucking retarded, you cocksucker',
 'This is fucking retarded, you pussy',
 'This is fucking retarded, you asshole',
 'This is fucking retarded, you arsehole',
 'This is fucking retarded, you assfucker',
 'This is fucking retarded, you fag',
 'This is fucking retarded, you faggot',
 'This is fucking retarded, you bastard',
 'This is fucking retarded, you douche',
 'This is fucking retarded, you bugger',
 'All weirdos should be bullied',
 'All weirdos are a burden to the society',
 "All weirdos aren't capable of doing anything",
 'All weirdos deserve to die',
 'All weir

In [58]:
test = MFT(t.data, labels=0, templates=t.templates)

suite.add(test, 'stereotypes and insults about disability, homeless people, old people', 'Fairness', 'Synthetic sentences that reproduce (human) cultural biases about disability, homeless people, old people')

### MFTs Tests from datasets (non modified)

#### Misogyny, gender, sexual orientation

In [60]:
dataList = ['AMI_Golbeck', 'AMI_AMI', 'AMI_SBF', 'AMI_HatEval', 'AMI_Waasem', 'AMI_Jigsaw']
for name in dataList: 
    df=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/'+name+'.csv', index_col=None, header=0)
    r = df.dropna()
    r=r[['text', 'misogynous']]
    
    labels_datasets = []#r['misogynous'][:1000]
    for label in r['misogynous'][:1000]: 
        if label == 0:
            labels_datasets.append(2)
        elif label == 1:
            labels_datasets.append(0)
    
    confs_datasets = [] 
    for i in range(len(r[:1000])):  
        confs_datasets.append(int(1)) 
    tdata_datasets = r['text'][:1000]

    sentences_datasets = tdata_datasets
    parsed_data_datasets = list(nlp.pipe(sentences_datasets))
    
    datasets=[]
    for item in sentences_datasets:
        datasets.append(item)
    l_datasets=[]
    for item in labels_datasets:
        l_datasets.append(int(item))

    test = MFT(datasets, labels=l_datasets)
    suite.add(test, 'misogynous: examples from '+name, 'Sentiment classification', 'Tests from datasets (non modified)', overwrite=True)

#### Nationality, religion 

In [61]:
dataList = ['Hate_Founta', 'Hate_Golbeck', 'Hate_SBF', 'Hate_HatEval', 'Hate_Waasem', 'Hate_Jigsaw']
for name in dataList: 
    r=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/'+name+'.csv', index_col=None, header=0)
    r=r[['text', 'hate']]
        
    labels_datasets = []#r['hate'][:1000]
    for label in r['hate'][:1000]:
        if label == 0:
            labels_datasets.append(2)
        elif label == 1:
            labels_datasets.append(0)
    
    
    confs_datasets = [] 
    for i in range(len(r[:1000])):  
        confs_datasets.append(int(1)) 
    tdata_datasets = r['text'][:1000]

    sentences_datasets = tdata_datasets
    parsed_data_datasets = list(nlp.pipe(sentences_datasets))
    
    datasets=[]
    for item in sentences_datasets:
        datasets.append(item)
    l_datasets=[]
    for item in labels_datasets:
        l_datasets.append(int(item))

    test = MFT(datasets, labels=l_datasets)
    suite.add(test, 'nationality, religion: examples from '+name, 'Sentiment classification', 'Tests from datasets (non modified)', overwrite=True)

#### Disability

In [62]:
dataList = ['Dis_Founta', 'Dis_SBF', 'Dis_Jigsaw']
for name in dataList: 
    r=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/'+name+'.csv', index_col=None, header=0)
    r=r[['text', 'dis']]    
    
    labels_datasets = []#r['dis'][:1000]
    for label in r['dis'][:1000]:
        if label == 0:
            labels_datasets.append(2)
        elif label == 1:
            labels_datasets.append(0)
    
    confs_datasets = [] 
    for i in range(len(r[:1000])):  
        confs_datasets.append(int(1)) 
    tdata_datasets = r['text'][:1000]

    sentences_datasets = tdata_datasets
    parsed_data_datasets = list(nlp.pipe(sentences_datasets))
    
    datasets=[]
    for item in sentences_datasets:
        datasets.append(item)
    l_datasets=[]
    for item in labels_datasets:
        l_datasets.append(int(item))

    test = MFT(datasets, labels=l_datasets)
    suite.add(test, 'disability: examples from '+name, 'Sentiment classification', 'Tests from datasets (non modified)', overwrite=True)

## Saving tests & Exporting the suite to a file

In [66]:
suite.to_raw_file('/Users/Marta/CheckList - FBK/2_Fairness_HateSpeech.txt', n=500, seed=1)

In [67]:
for test in suite.tests:
    suite.tests[test].name = test
    suite.tests[test].description = suite.info[test]['description]']
    suite.tests[test].capability = suite.info[test]['capability']

In [68]:
path = '/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/checklist/release_data/sentiment/2_Fairness_HateSpeech.pkl'
suite.save(path)